### Algorytmy Macierzowe lab 1

#### Konfiguracja

In [201]:
import numpy as np
from math import log, ceil, pow
from typing import Tuple

#### Generowanie macierzy

In [202]:
def generate_matrix(ncols: int, nrows: int):
    return np.random.uniform(low=0.00000001, high=1.0, size=(nrows, ncols))


#### Licznik operacji

In [203]:
class Counter:
    def __init__(self) -> None:
        self.add_counter = 0
        self.mul_counter = 0
        self.div_counter = 0
        self.sub_counter = 0
        self.operation_counter = 0
    
    def add(self, A: np.ndarray , B: np.ndarray) -> np.ndarray:
        self.add_counter += A.size
        self.operation_counter += A.size
        return A + B

    def mul(self, A: np.ndarray , B: np.ndarray) -> np.ndarray:
        self.mul_counter += A.size
        self.operation_counter += A.size
        return A * B
    
    def div(self, A: np.ndarray , B: np.ndarray) -> np.ndarray:
        self.div_counter += A.size
        self.operation_counter += A.size
        return A / B

    def sub(self, A: np.ndarray , B: np.ndarray) -> np.ndarray:
        self.sub_counter += A.size
        self.operation_counter += A.size
        return A - B
    
    def print_counters(self) -> None:
        print(f'Number of add operations: {self.add_counter}')
        print(f'Number of sub operations: {self.sub_counter}')
        print(f'Number of div operations: {self.div_counter}')
        print(f'Number of mul operations: {self.mul_counter}')
        print(f'Number of operations: {self.operation_counter}')
        

#### Funckje pomocniczne

In [204]:
def resize_matrix_to_2n(A: np.ndarray, B: np.ndarray): 
    """Change size of matrix to simplify processing"""
    size_A = A.shape
    size_B = B.shape
    new_height: int = 0
    new_width: int = 0

    if size_A[0] > size_B[0]:
        new_height = find_next_power_of_2(size_A[0])
    else:
        new_height = find_next_power_of_2(size_B[0])
    
    if size_A[1] > size_B[1]:
        new_width = find_next_power_of_2(size_A[1])
    else:
        new_width = find_next_power_of_2(size_B[1])

    new_A = np.pad(A, [(0, new_height - size_A[0]), (0, new_width - size_A[1])], mode='constant')
    new_B = np.pad(B, [(0, new_height - size_B[0]), (0, new_width - size_B[1])], mode='constant')
    return new_A, new_B

def split(array: np.ndarray, nrows: int, ncols: int) -> Tuple:
    """Split a matrix into sub-matrices"""
    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))


def find_next_power_of_2(number: int) -> int:
    """Finds closest number that is power of 2"""
    return int(pow(2, ceil(log(number)/log(2))))

#### Algorytm Binet'a

In [205]:
A = generate_matrix(5, 5)
B = generate_matrix(5, 5)

def binet_core_algorithm(A: np.ndarray, B: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add
    sub = counter.sub
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0]//2
        A11, A12, A21, A22 = split(A, split_at, split_at)
        B11, B12, B21, B22 = split(B, split_at, split_at)
        
        C11 = add(binet_core_algorithm(A11, B11, counter), binet_core_algorithm(A12, B21, counter))
        C12 = add(binet_core_algorithm(A11, B12, counter), binet_core_algorithm(A12, B22, counter))
        C21 = add(binet_core_algorithm(A21, B11, counter), binet_core_algorithm(A22, B21, counter))
        C22 = add(binet_core_algorithm(A21, B12, counter), binet_core_algorithm(A22, B22, counter))
        
        return np.concatenate(
            [np.concatenate([C11, C12], axis = 1),
            np.concatenate([C21, C22],axis = 1)],
             axis = 0)
    else:
        return mul(A, B)

def binet_algorithm(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    counter = Counter()
    new_A, new_B = resize_matrix_to_2n(A, B)
    C = binet_core_algorithm(new_A, new_B, counter)
    counter.print_counters()
    C = C[~np.all(C == 0, axis=1)]
    C = C[:, ~np.all(C == 0, axis=0)]
    return C

print(binet_algorithm(A, B))
print()
print(A@B)

Number of add operations: 448
Number of sub operations: 0
Number of div operations: 0
Number of mul operations: 512
Number of operations: 960
[[1.63826614 0.9521745  2.2535109  0.88607742 1.5560085 ]
 [1.88106317 1.69791086 2.81468099 1.35424403 2.01299766]
 [1.32334441 1.53463061 2.19393697 1.33354818 1.7536106 ]
 [0.65309173 0.55055134 0.97295119 0.49181014 0.60622932]
 [1.32119378 1.04539526 1.79527033 0.82762042 1.23242526]]

[[1.63826614 0.9521745  2.2535109  0.88607742 1.5560085 ]
 [1.88106317 1.69791086 2.81468099 1.35424403 2.01299766]
 [1.32334441 1.53463061 2.19393697 1.33354818 1.7536106 ]
 [0.65309173 0.55055134 0.97295119 0.49181014 0.60622932]
 [1.32119378 1.04539526 1.79527033 0.82762042 1.23242526]]


#### Algorytm Strassen'a
https://en.wikipedia.org/wiki/Strassen_algorithm
Wszystkie oznaczenia tak jak tu

In [206]:
A = generate_matrix(2, 2)
B = generate_matrix(2, 2)

def strassen_core_algorith(A: np.ndarray, B: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add
    sub = counter.sub
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0]//2
        A11, A12, A21, A22 = split(A, split_at, split_at)
        B11, B12, B21, B22 = split(B, split_at, split_at)
        
        M1 = strassen_core_algorith(add(A11, A22), add(B11, B22), counter)
        M2 = strassen_core_algorith(add(A21, A22), B11, counter)
        M3 = strassen_core_algorith(A11, sub(B12, B22), counter)
        M4 = strassen_core_algorith(A22, sub(B21, B11), counter)
        M5 = strassen_core_algorith(add(A11, A12), B22, counter)
        M6 = strassen_core_algorith(sub(A21, A11), add(B11, B12), counter)
        M7 = strassen_core_algorith(sub(A12, A22), add(B21, B22), counter)    

        C11 = add(sub(add(M1, M4), M5), M7)
        C12 = add(M3, M5)
        C21 = add(M2, M4)
        C22 = add(add(sub(M1, M2), M3), M6)

        return np.concatenate(
            [np.concatenate([C11, C12], axis = 1),
            np.concatenate([C21, C22],axis = 1)],
             axis = 0)
    else:
        return mul(A, B)

def strassen_algorith(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    counter = Counter()
    new_A, new_B = resize_matrix_to_2n(A, B)
    C = strassen_core_algorith(new_A, new_B, counter)
    counter.print_counters()
    C = C[~np.all(C == 0, axis=1)]
    C = C[:, ~np.all(C == 0, axis=0)]
    return C

print(strassen_algorith(A, B))
print()
print(A@B)

Number of add operations: 12
Number of sub operations: 6
Number of div operations: 0
Number of mul operations: 7
Number of operations: 25
[[0.31480336 0.45772077]
 [0.55088515 0.33534832]]

[[0.31480336 0.45772077]
 [0.55088515 0.33534832]]


In [207]:
A = generate_matrix(2, 2)
B = generate_matrix(2, 2)

def w_core_algorithm(A: np.ndarray, B: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add
    sub = counter.sub
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0]//2
        A11, A12, A21, A22 = split(A, split_at, split_at)
        B11, B12, B21, B22 = split(B, split_at, split_at)
        C_A = sub(B12, B11)
        C_D = add(A21, A22)
        A_A = w_core_algorithm(A11, B11, counter)

        U = w_core_algorithm(sub(A21, A11), sub(B12, B22), counter)
        V = w_core_algorithm(C_D, C_A, counter)
        W = add(A_A,
        w_core_algorithm(sub(C_D, A11), sub(B22, C_A), counter))
        W_U = add(W, U)

        C11 =  add(A_A, w_core_algorithm(A12, B21, counter))
        C12 = add(add(W, V), w_core_algorithm(sub(add(A11, A12), C_D), B22, counter))
        C21 = add(W_U, w_core_algorithm(A22, sub(add(B21, C_A), B22), counter))
        C22 = add(W_U, V)
        
        return np.concatenate(
            [np.concatenate([C11, C12], axis = 1),
            np.concatenate([C21, C22],axis = 1)],
             axis = 0)
    else:
        return mul(A, B)

def winograd_algorith(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    counter = Counter()
    new_A, new_B = resize_matrix_to_2n(A, B)
    C = w_core_algorithm(new_A, new_B, counter)
    counter.print_counters()
    C = C[~np.all(C == 0, axis=1)]
    C = C[:, ~np.all(C == 0, axis=0)]
    return C

print(winograd_algorith(A, B))
print()
print(A@B)

Number of add operations: 10
Number of sub operations: 7
Number of div operations: 0
Number of mul operations: 7
Number of operations: 24
[[0.56624682 0.47870243]
 [0.63604975 0.60723356]]

[[0.56624682 0.47870243]
 [0.63604975 0.60723356]]
